In [1]:
import transformers

In [2]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
model_id="gpt2"
tokenizer=AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

C:\Users\PRASANNA P\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PRASANNA P\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\PRASANNA P\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: F

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
tokenizer.model_max_length

1024

In [5]:
tokenizer.vocab_size

50257

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
model=AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [13]:
def test_prompt(prompt,yourmodel):
    input_id=tokenizer(prompt,return_tensors='pt')
    logits=yourmodel.generate(**input_id.to(device),max_new_tokens=50)
    return tokenizer.decode(logits[0])


In [14]:
text="Photonic chips are "
output=test_prompt(text,model)
print(output)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Photonic chips are  a very common type of chip. They are used to create a digital image. They are used to create a digital image. They are used to create a digital image. They are used to create a digital image. They are used to create a


In [15]:
from transformers import (
    Trainer,
    DataCollatorForLanguageModeling,
    TrainingArguments
)

from datasets import load_dataset,load_metric, Dataset

In [16]:
import pandas as pd

In [20]:
import json

data=[]

with open('C:/Users/PRASANNA P/Desktop/AI code/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features=json.loads(line)
        if features["context"]:
            continue
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

In [21]:
data[0]

'Instruction:\nWhich is a species of fish? Tope or Rope\n\nResponse:\nTope'

In [27]:
data_500 = [x for x in data if len(x)<=500]

In [28]:
len(data_500)

7339

In [29]:
data_500[0:5]

['Instruction:\nWhich is a species of fish? Tope or Rope\n\nResponse:\nTope',
 'Instruction:\nWhy can camels survive for long without water?\n\nResponse:\nCamels use the fat in their humps to keep them filled with energy and hydration for long periods of time.',
 "Instruction:\nAlice's parents have three daughters: Amy, Jessy, and what’s the name of the third daughter?\n\nResponse:\nThe name of the third daughter is Alice",
 'Instruction:\nWho gave the UN the land in NY to build their HQ\n\nResponse:\nJohn D Rockerfeller',
 'Instruction:\nWhy mobile is bad for human\n\nResponse:\nWe are always engaged one phone which is not good.']

In [30]:
data=data_500[:300]

In [31]:
len(data)

300

In [33]:
tokenizer(data[0])


{'input_ids': [6310, 2762, 25, 198, 13828, 318, 257, 4693, 286, 5916, 30, 309, 3008, 393, 371, 3008, 198, 198, 31077, 25, 198, 51, 3008], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [34]:
json_ds = Dataset.from_pandas(pd.DataFrame(data=data))


In [35]:
def tokenize_instruction(row):
    # print(row)
    input_token = tokenizer(row['0'], padding="max_length",
                            truncation=True, max_length=512)
    input_token['labels'] = input_token['input_ids'].copy()
    return input_token

In [36]:
tokenizer.pad_token = tokenizer.eos_token
dataset_tokenized = json_ds.map(tokenize_instruction,
                                remove_columns=["0"],
                                batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [37]:
process_ds=dataset_tokenized.train_test_split(test_size=0.2)

In [38]:
process_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 240
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 60
    })
})

In [57]:
process_ds.with_format("torch")

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 240
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 60
    })
})

In [40]:
process_ds['train'][1]

{'input_ids': [6310,
  2762,
  25,
  198,
  24446,
  502,
  266,
  6750,
  777,
  389,
  4736,
  11,
  393,
  2585,
  30,
  14639,
  11,
  442,
  4549,
  11,
  3936,
  11,
  38433,
  756,
  11,
  2669,
  11,
  2669,
  6257,
  11,
  5336,
  474,
  577,
  11,
  15489,
  1989,
  198,
  198,
  31077,
  25,
  198,
  17402,
  6257,
  11,
  4842,
  11,
  39587,
  756,
  11,
  2986,
  474,
  577,
  389,
  4736,
  11,
  290,
  14639,
  11,
  3936,
  11,
  2669,
  389,
  1294,
  2585,
  13,
  4696,
  1989,
  318,
  6159,
  257,
  1181,
  4249,
  257,
  1748,
  13,
  632,
  318,
  9759,
  284,
  262,
  1989,
  287,
  7840,
  7257,
  543,
  10874,
  286,
  3294,
  4736,
  13,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  5

In [41]:
len(process_ds['train'][12]['input_ids'])

512

In [58]:
model.get_memory_footprint()/1024**3

0.47529320418834686

In [59]:
steps_per_epoch = len(process_ds['train'])//8
steps_per_epoch

30

In [60]:
args=TrainingArguments(
    per_device_eval_batch_size=2,
    per_device_train_batch_size=2,
    output_dir='C:/Users/PRASANNA P/Desktop/AI code',
    eval_strategy="steps",
    logging_steps=1,
    eval_steps=steps_per_epoch,
    save_strategy="epoch",
    num_train_epochs=5,
    lr_scheduler_type="constant",
    learning_rate=2e-5,
    group_by_length=True,
    push_to_hub=False,
    report_to="none"
)

In [61]:
datacollator=DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=False)

In [62]:
trainer=Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=datacollator,
    train_dataset=process_ds["train"],
    eval_dataset=process_ds["test"],
    args=args

)

In [63]:
trainer.args.device

device(type='cuda', index=0)

In [64]:
trainer.train()

  0%|          | 0/600 [00:00<?, ?it/s]

{'loss': 2.1556, 'grad_norm': 21.4136962890625, 'learning_rate': 2e-05, 'epoch': 0.01}
{'loss': 1.3808, 'grad_norm': 15.603765487670898, 'learning_rate': 2e-05, 'epoch': 0.02}
{'loss': 1.8932, 'grad_norm': 12.143841743469238, 'learning_rate': 2e-05, 'epoch': 0.03}
{'loss': 2.7959, 'grad_norm': 17.93763542175293, 'learning_rate': 2e-05, 'epoch': 0.03}
{'loss': 2.5221, 'grad_norm': 20.46424102783203, 'learning_rate': 2e-05, 'epoch': 0.04}
{'loss': 2.4479, 'grad_norm': 17.580219268798828, 'learning_rate': 2e-05, 'epoch': 0.05}
{'loss': 2.2474, 'grad_norm': 20.20660972595215, 'learning_rate': 2e-05, 'epoch': 0.06}
{'loss': 2.0592, 'grad_norm': 13.399787902832031, 'learning_rate': 2e-05, 'epoch': 0.07}
{'loss': 2.2353, 'grad_norm': 17.801496505737305, 'learning_rate': 2e-05, 'epoch': 0.07}
{'loss': 2.2123, 'grad_norm': 15.694369316101074, 'learning_rate': 2e-05, 'epoch': 0.08}
{'loss': 2.6668, 'grad_norm': 16.302753448486328, 'learning_rate': 2e-05, 'epoch': 0.09}
{'loss': 2.0675, 'grad_nor

  0%|          | 0/30 [00:00<?, ?it/s]

{'eval_loss': 2.5136733055114746, 'eval_runtime': 23.7404, 'eval_samples_per_second': 2.527, 'eval_steps_per_second': 1.264, 'epoch': 0.25}
{'loss': 2.3156, 'grad_norm': 16.008684158325195, 'learning_rate': 2e-05, 'epoch': 0.26}
{'loss': 1.9297, 'grad_norm': 15.080085754394531, 'learning_rate': 2e-05, 'epoch': 0.27}
{'loss': 2.4831, 'grad_norm': 13.927925109863281, 'learning_rate': 2e-05, 'epoch': 0.28}
{'loss': 2.27, 'grad_norm': 16.22213363647461, 'learning_rate': 2e-05, 'epoch': 0.28}
{'loss': 1.5559, 'grad_norm': 11.822734832763672, 'learning_rate': 2e-05, 'epoch': 0.29}
{'loss': 2.5642, 'grad_norm': 18.90094566345215, 'learning_rate': 2e-05, 'epoch': 0.3}


KeyboardInterrupt: 